# Generate job files for MSI computing
Main feature is to get all subfolders in a directory, and set up the output folders correspondingly

In [1]:
import myImageLib as mil
import os
import numpy as np
from tool_utils import header

Generate a header for job file:
```Python
def header(walltime, mem, code_path, to_file=None):
    """
    walltime -- estimate running time (hr)
    mem -- memory needed (gb)
    code_path -- the folder in which the code can be found
    to_file -- file object to write to. If you want to print in console, set to None (default)
```

In [4]:
# test header()
header(8, 3, '~/code/Python/Correlation/src/py_files')

#!/bin/bash -l
#PBS -l walltime=8:00:00,nodes=1:ppn=8,mem=3gb
#PBS -m abe
#PBS -M liux3141@umn.edu

cd ~/code/Python/Correlation/src/py_files
module load python3
source activate pythonEnv


- 00.pbs: python to_tif nd2Dir
- 01.pbs: python piv_imseq.py img_folder output_folder winsize overlap fps smooth
- 02.pbs: python df2_kinetics.py img_folder kinetics_folder seg_length
- 03.pbs: python overall_intensity.py folder folder_out
- 04.pbs: python corr_imseq.py input_folder output_folder wsize step


- 05.pbs: python div2.py piv_folder imgDir output_folder winsize step
- 06.pbs: python cav_imseq.py piv_folder output_folder
- 07.pbs: python dc_adv.py folder_img folder_piv folder_out interval fps step
- 08.pbs: python div_x_dcadv.py folder_div folder_dcadv folder_out

In [25]:
# Global dirs
folder = r'E:\Github\Python\job_files\08102020' # job file folder
if os.path.exists(folder) == False:
    os.makedirs(folder)
data_path = '~/Data/08052020'
num = range(0, 12)
fpsL = [30, 30, 30, 30, 30, 30, 30, 30, 30, 10, 10, 10]

In [46]:
nd2Dir = data_path + '/{0:02d}.nd2'
img_folder = data_path + '/{0:02d}/8-bit'
piv_folder = data_path + '/piv_imseq/{0:02d}'
i_folder = data_path + '/overall_intensity/{0:02d}'
dxd_folder = data_path + '/div_x_dcadv/{0:02d}'
kinetics_folder = data_path + '/df2_kinetics/{0:02d}'
dcadv_folder = data_path + '/dc_adv/{0:02d}/dt={1:d}'
corr_folder = data_path + '/corr_imseq/{0:02d}'
cav_folder = data_path + '/cav_imseq/{0:02d}'
div_folder = data_path + '/div2/{0:02d}'

In [20]:
nd2Dir.format(1)

'~/Data/08042020/01.nd2'

In [28]:
script = 'to_tif.py'
job_num = 0 # job file number
walltime = 3 # hours
mem = 3 # gb
code_dir = '~/code/Python/generic_proc/py_files' # python script dir
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(('python ' + script + ' ' + nd2Dir +' &\n').format(n))
    f.write('wait')

In [35]:
script = 'piv_imseq.py'
job_num = 1 # job file number
walltime = 20 # hours
mem = 3 # gb
code_dir = '~/code/Python/PIV/src/py_files' # python script dir
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(('python ' + script + ' ' + img_folder + ' ' + piv_folder + ' 50 25 1' +' &\n').format(n))
    f.write('wait')

In [37]:
script = 'df2_kinetics.py'
job_num = 2 # job file number
walltime = 10 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
cmd = ('python ' + script + ' ' + img_folder + ' ' + kinetics_folder + ' 50' +' &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(cmd.format(n))
    f.write('wait')

In [38]:
# python overall_intensity.py folder folder_out
script = 'overall_intensity.py'
job_num = 3 # job file number
walltime = 3 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
cmd = ('python ' + script + ' ' + img_folder + ' ' + i_folder +' &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(cmd.format(n))
    f.write('wait')

In [43]:
# python corr_imseq.py img_folder corr_folder wsize step
script = 'corr_imseq.py'
job_num = 4 # job file number
walltime = 16 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
cmd = ('python ' + script + ' ' + img_folder + ' ' + corr_folder + ' 25 25' +' &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(cmd.format(n))
    f.write('wait')

In [44]:
# python div2.py piv_folder imgDir output_folder winsize step
script = 'div2.py'
job_num = 5 # job file number
walltime = 16 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
cmd = ('python ' + script + ' ' + piv_folder + ' ' + img_folder + ' ' + div_folder + ' 50 25' +' &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(cmd.format(n))
    f.write('wait')

In [45]:
# python cav_imseq.py piv_folder output_folder
script = 'cav_imseq.py'
job_num = 6 # job file number
walltime = 5 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
cmd = ('python ' + script + ' ' + piv_folder + ' ' + cav_folder +' &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(cmd.format(n))
    f.write('wait')

In [47]:
# python dc_adv.py folder_img folder_piv folder_out interval fps step
script = 'dc_adv.py'
job_num = 7 # job file number
walltime = 16 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
cmd = ('python ' + script + ' ' + img_folder + ' ' + piv_folder + ' ' + dcadv_folder + ' {1:d} {2:d} 25' +' &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(cmd.format(n, 2*fps, fps))
    f.write('wait')

In [48]:
# python div_x_dcadv.py folder_div folder_dcadv folder_out
script = 'div_x_dcadv.py'
job_num = 8 # job file number
walltime = 16 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
cmd = ('python ' + script + ' ' + div_folder + ' ' + dcadv_folder + ' ' + dxd_folder + ' &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(cmd.format(n, 2*fps, fps))
    f.write('wait')

In [7]:
# piv, 01
folder = r'E:\Github\Python\job_files\08082020'
# folder_out = '~/numerics/illumination_noise'
if os.path.exists(folder) == False:
    os.makedirs(folder)
with open(os.path.join(folder, '01.pbs'), 'w') as f:
    header(16, 3, '~/code/Python/PIV/src/py_files', to_file=f)
    num = range(0, 12)
    fpsL = [30, 30, 30, 30, 30, 30, 30, 30, 30, 10, 10, 10]
    for n, fps in zip(num, fpsL):
        f.write('python piv_imseq.py ~/Data/08042020/{0:02d}/8-bit ~/Data/08042020/piv_imseq/{0:02d}  50 25 {1:d} 1 &\n'.format(n, fps))
    f.write('wait')

In [9]:
# df2_kinetics, 02
folder = r'E:\Github\Python\job_files\08082020'
# folder_out = '~/numerics/illumination_noise'
if os.path.exists(folder) == False:
    os.makedirs(folder)
with open(os.path.join(folder, '02.pbs'), 'w') as f:
    header(16, 3, '~/code/Python/Correlation/src/py_files', to_file=f)
    num = range(0, 12)
    fpsL = [30, 30, 30, 30, 30, 30, 30, 30, 30, 10, 10, 10]
    for n, fps in zip(num, fpsL):
        f.write('python df2_kinetics.py ~/Data/08042020/{0:02d}/8-bit ~/Data/08042020/df2_kinetics/{0:02d} 50 &\n'.format(n))
    f.write('wait')

In [10]:
# overall_intensity, 03
folder = r'E:\Github\Python\job_files\08082020'
# folder_out = '~/numerics/illumination_noise'
if os.path.exists(folder) == False:
    os.makedirs(folder)
with open(os.path.join(folder, '03.pbs'), 'w') as f:
    header(16, 3, '~/code/Python/Correlation/src/py_files', to_file=f)
    num = range(0, 12)
    fpsL = [30, 30, 30, 30, 30, 30, 30, 30, 30, 10, 10, 10]
    for n, fps in zip(num, fpsL):
        f.write('python overall_intensity.py ~/Data/08042020/{0:02d}/8-bit ~/Data/08042020/overall_intensity/{0:02d} &\n'.format(n))
    f.write('wait')

In [16]:
# corr_imseq, 04
folder = r'E:\Github\Python\job_files\08082020'
# folder_out = '~/numerics/illumination_noise'
if os.path.exists(folder) == False:
    os.makedirs(folder)
with open(os.path.join(folder, '04.pbs'), 'w') as f:
    header(16, 3, '~/code/Python/Correlation/src/py_files', to_file=f)
    num = range(0, 12)
    fpsL = [30, 30, 30, 30, 30, 30, 30, 30, 30, 10, 10, 10]
    for n, fps in zip(num, fpsL):
        f.write('python corr_imseq.py ~/Data/08042020/{0:02d}/8-bit ~/Data/08042020/corr_imseq/{0:02d} 10 10 &\n'.format(n))
    f.write('wait')

In [12]:
# div2, 05
folder = r'E:\Github\Python\job_files\08082020'
# folder_out = '~/numerics/illumination_noise'
if os.path.exists(folder) == False:
    os.makedirs(folder)
with open(os.path.join(folder, '05.pbs'), 'w') as f:
    header(16, 3, '~/code/Python/Correlation/src/py_files', to_file=f)
    num = range(0, 12)
    fpsL = [30, 30, 30, 30, 30, 30, 30, 30, 30, 10, 10, 10]
    for n, fps in zip(num, fpsL):
        f.write('python div2.py ~/Data/08042020/piv_imseq/{0:02d} ~/Data/08042020/{0:02d}/8-bit ~/Data/08032020/div2/{0:02d} 50 25 &\n'.format(n))
    f.write('wait')

In [13]:
# cav_imseq, 06
folder = r'E:\Github\Python\job_files\08082020'
# folder_out = '~/numerics/illumination_noise'
if os.path.exists(folder) == False:
    os.makedirs(folder)
with open(os.path.join(folder, '06.pbs'), 'w') as f:
    header(16, 3, '~/code/Python/Correlation/src/py_files', to_file=f)
    num = range(0, 12)
    fpsL = [30, 30, 30, 30, 30, 30, 30, 30, 30, 10, 10, 10]
    for n, fps in zip(num, fpsL):
        f.write('python cav_imseq.py ~/Data/08042020/piv_imseq/{0:02d} ~/Data/08042020/cav_imseq/{0:02d} &\n'.format(n))
    f.write('wait')

In [14]:
# dc_adv, 07
folder = r'E:\Github\Python\job_files\08082020'
# folder_out = '~/numerics/illumination_noise'
if os.path.exists(folder) == False:
    os.makedirs(folder)
with open(os.path.join(folder, '07.pbs'), 'w') as f:
    header(16, 3, '~/code/Python/Correlation/src/py_files', to_file=f)
    num = range(0, 12)
    fpsL = [30, 30, 30, 30, 30, 30, 30, 30, 30, 10, 10, 10]
    for n, fps in zip(num, fpsL):
        f.write('python dc_adv.py ~/Data/08042020/{0:02d}/8-bit ~/Data/08042020/piv_imseq/{0:02d} ~/Data/08042020/dc_adv/{0:02d}/dt={1:d} {1:d} {2:d} 25 &\n'.format(n, 2*fps, fps))
    f.write('wait')

In [15]:
# div_x_dcadv, 08, after 07 finishes
folder = r'E:\Github\Python\job_files\08082020'
# folder_out = '~/numerics/illumination_noise'
if os.path.exists(folder) == False:
    os.makedirs(folder)
with open(os.path.join(folder, '08.pbs'), 'w') as f:
    header(16, 3, '~/code/Python/Correlation/src/py_files', to_file=f)
    num = range(0, 12)
    fpsL = [30, 30, 30, 30, 30, 30, 30, 30, 30, 10, 10, 10]
    for n, fps in zip(num, fpsL):
        f.write('python div_x_dcadv.py ~/Data/08042020/div2/{0:02d} ~/Data/08042020/dc_adv/{0:02d} ~/Data/08042020/div_x_dcadv/{0:02d} &\n'.format(n))
    f.write('wait')

In [4]:
# df2_nobp, 09
folder = r'E:\Github\Python\job_files\08082020'
if os.path.exists(folder) == False:
    os.makedirs(folder)
with open(os.path.join(folder, '09.pbs'), 'w') as f:
    header(16, 3, '~/code/Python/Correlation/src/py_files', to_file=f)
    num = range(0, 12)
    fpsL = [30, 30, 30, 30, 30, 30, 30, 30, 30, 10, 10, 10]
    for n, fps in zip(num, fpsL):
        f.write('python df2_nobp.py ~/Data/08042020/{0:02d}/8-bit ~/Data/08042020/df2_nobp/{0:02d} &\n'.format(n))
    f.write('wait')

'1'